In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services. Name of the environmental variables is arbitraray because langrila modules accepts that name as an argument.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# Import modules

In [3]:
from langrila import Agent, Tool
from langrila.anthropic import AnthropicClient
from langrila.google import GoogleClient
from langrila.openai import OpenAIClient

# Define tools

In this example, we use dummy tools as shown below. In langrila, tool's schema is generated from the tool's type hint and docstring in default, so it's important to write in details. Here is an example using dummy tools.

Tool can have a pydantic model in its arguments. Nested pydantic model is also acceptable.

In [4]:
import random
from enum import Enum

from pydantic import BaseModel, Field


class MusicGenre(str, Enum):
    rock = "rock"
    pop = "pop"
    jazz = "jazz"
    classical = "classical"
    hip_hop = "hip-hop"


class EqualizerSettings(BaseModel):
    hz_100: int = Field(0, ge=-12, le=12, description="The 100 Hz band in dB.")
    hz_200: int = Field(0, ge=-12, le=12, description="The 200 Hz band in dB.")
    hz_400: int = Field(0, ge=-12, le=12, description="The 400 Hz band in dB.")
    hz_800: int = Field(0, ge=-12, le=12, description="The 800 Hz band in dB.")
    hz_1600: int = Field(0, ge=-12, le=12, description="The 1600 Hz band in dB.")
    hz_3200: int = Field(0, ge=-12, le=12, description="The 3200 Hz band in dB.")
    hz_6400: int = Field(0, ge=-12, le=12, description="The 4000 Hz band in dB.")


def start_music(genre: MusicGenre) -> str:
    """
    Start playing a random music with the given the genre.

    Parameters
    ----------
    genre : MusicGenre
        The genre of the music to play.

    Returns
    ----------
    str
        A message indicating that the music has started.
    """

    bpms = {
        MusicGenre.rock: 150,
        MusicGenre.pop: 110,
        MusicGenre.jazz: 120,
        MusicGenre.classical: 76,
        MusicGenre.hip_hop: 100,
    }

    return (
        f"Starting music! Genre: {genre}, "
        f"Volume: {random.uniform(0.0, 1.0)}, "
        f"BPM: {bpms[genre]}"
    )


def change_music_volume(volume: float) -> str:
    """
    Change the music volume.

    Parameters
    ----------
    volume : float
        The new volume to set. It should be between 0.0 and 1.0.

    Returns
    ----------
    str
        A message indicating that the volume has been changed.
    """
    return f"Changing volume to {volume}"


def change_music(genre: MusicGenre) -> str:
    """
    Change the music.

    Parameters
    ----------
    genre : MusicGenre
        The genre of the music to change to.

    Returns
    ----------
    str
        A message indicating that the music has been changed.
    """
    bpm_ranges = {
        MusicGenre.rock: (120, 180),
        MusicGenre.pop: (80, 140),
        MusicGenre.jazz: (70, 120),
        MusicGenre.classical: (70, 100),
        MusicGenre.hip_hop: (80, 120),
    }

    new_bpm = random.randint(bpm_ranges[genre][0], bpm_ranges[genre][1])

    return f"Turning to another music! Genre: {genre}, BPM: {new_bpm}"


def change_equalizer_settings(settings: EqualizerSettings) -> str:
    """
    Change the equalizer settings of the music.

    Parameters
    ----------
    settings : EqualizerSettings
        The new equalizer settings to set.

    Returns
    ----------
    str
        A message indicating that the equalizer settings have been changed.
    """
    return f"Changing equalizer settings to {settings.model_dump(exclude_none=True, exclude_unset=True)}"

# Instantiating Agent

Client modules

In [5]:
# For OpenAI
openai_client = OpenAIClient(api_key_env_name="OPENAI_API_KEY")

# For Azure OpenAI
azure_openai_client = OpenAIClient(
    api_key_env_name="AZURE_API_KEY",
    api_type="azure",
    azure_api_version="2024-11-01-preview",
    azure_endpoint_env_name="AZURE_ENDPOINT",
    azure_deployment_id_env_name="AZURE_DEPLOYMENT_ID",
)

# For Gemini on Google AI Studio
google_dev_client = GoogleClient(
    api_key_env_name="GEMINI_API_KEY",
)

# For Gemini on Google Cloud VertexAI
vertexai_client = GoogleClient(
    api_type="vertexai",
    project_id_env_name="GOOGLE_CLOUD_PROJECT",
    location="us-central1",
)

# For Claude of Anthropic
anthropic_client = AnthropicClient(
    api_key_env_name="ANTHROPIC_API_KEY",
)

# For Claude of Amazon Bedrock
claude_bedrock_client = AnthropicClient(
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
)

Tools must be specified when the agent is initialized.

In [6]:
tools = [start_music, change_music_volume, change_music, change_equalizer_settings]

In [7]:
openai_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=tools,
)

# Generating text with tool calling

In [8]:
prompt = (
    "Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM."
)

## Synchronous, non-streaming text generation

In [9]:
response = openai_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-09 23:41:15] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:41:15] INFO | root: Generating text
[2025-01-09 23:41:16] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre":"jazz"}', call_id='call_trYO0Umbb3LOJlVjrMK6K17g')]
[2025-01-09 23:41:16] INFO | Running tool: start_music
[2025-01-09 23:41:16] INFO | Tool: start_music successfully ran.
[2025-01-09 23:41:16] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.8590609162851207, BPM: 120', error=None, call_id='call_trYO0Umbb3LOJlVjrMK6K17g', args='{"genre":"jazz"}', name='start_music')]
[2025-01-09 23:41:16] INFO | root: Generating text
[2025-01-09 23:41:16] DEBUG | Response: [ToolCallResponse(name='change_music_volume', args='{"volume":0.2}', call_id='call_76oEjk02khr5UIsJe7qU87hk')]
[2025-01-09 23:41:16] INFO | Running tool: change_music_volume
[2025-01-09 23:41:16] INFO | Tool: ch

I've started playing jazz music at a low volume. The current BPM (Beats Per Minute) is 120. Enjoy the music!


Agent finally generates text response. Steps the agent generates final answer is like: 

1. Feed user's input
2. Call tools
3. Validate args with the pydantic schema validator
4. Actually run called tools
5. Generate text response based on the tool calling results

You can specify `planning` argument when the agent is instantiated. If `planning` is True, the agent makes a plan to answer the user's input based on the given tools, and then generates answer.

In [10]:
openai_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=tools,
    planning=True,  # <- Enable planning
)

In [11]:
response = openai_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-09 23:41:17] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.\n\nCapabilities:\n- start_music: Start playing a random music with the given the genre.\n- change_music_volume: Change the music volume.\n- change_music: Change the music.\n- change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-09 23:41:17] INFO | root: Generating text
[2025-01-09 23:41:22] DEBUG | Response: [TextResponse(text='### Plan to Address the Requirement\n\n1. **Start Playing Jazz Music**: Use the `start_music` tool to begin playing jazz music.\n2. **Adjust Volume**: Use the `change_music_vol

The jazz music has started playing at a low volume, and the estimated BPM for the track is around 120. Enjoy the music!


By planning, `start_music` tool was called at the first step, then `change_bpm ` tool and `change_music` tool was.

Here is the default planning prompt.

In [12]:
from langrila import AgentConfig

print(AgentConfig().internal_prompt.planning)

Please make a concise plan to answer the following question/requirement, considering the conversation history.
You can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.
Agent has no description while the tools have a description.

Question/Requirement:
{user_input}

Capabilities:
{capabilities}


This AgentConfig is configurable and customized config can be passes to the agent when instantiating as mentioned in [01.introduction.ipynb](./01.introduction.ipynb).

## Asynchronous, non-streaming text generation

In [13]:
response = await openai_agent.generate_text_async(prompt)

print(response.contents[0].text)

[2025-01-09 23:41:24] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.\n\nCapabilities:\n- start_music: Start playing a random music with the given the genre.\n- change_music_volume: Change the music volume.\n- change_music: Change the music.\n- change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-09 23:41:24] INFO | root: Generating text
[2025-01-09 23:41:28] DEBUG | Response: [TextResponse(text='### Plan to Address the Requirement\n\n1. **Start Playing Jazz Music**: Use the `start_music` tool to begin playing jazz music.\n2. **Set Volume to Low**: Use the `change_music

The jazz music has started playing at a low volume. The estimated BPM for the jazz music is around 120. Enjoy the music!


## Synchronous, streaming text generation

In [14]:
from langrila import TextResponse, ToolCallResponse
# from langrila.core.logger import DEFAULT_LOGGER as default_logger

In [15]:
streamed_response = openai_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-09 23:41:30] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.\n\nCapabilities:\n- start_music: Start playing a random music with the given the genre.\n- change_music_volume: Change the music volume.\n- change_music: Change the music.\n- change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-09 23:41:30] INFO | root: Generating text
[2025-01-09 23:41:34] DEBUG | Response: [TextResponse(text='### Plan to Address the Requirement\n\n1. **Start Playing Jazz Music**: Use the `start_music` tool to begin playing jazz music.\n2. **Adjust Volume**: Use the `change_music_vol

{"ge
{"genre":
{"genre": "jazz
{"genre": "jazz"}
{"vo
{"volume"
{"volume": 0.2}


[2025-01-09 23:41:35] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='call_LJKjbpgM80Xtnjh32cNWJoUb'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='call_Yh0n9Jjalh9XIRFqNpt4YEPh')]
[2025-01-09 23:41:35] INFO | Running tool: start_music
[2025-01-09 23:41:35] INFO | Tool: start_music successfully ran.
[2025-01-09 23:41:35] INFO | Running tool: change_music_volume
[2025-01-09 23:41:35] INFO | Tool: change_music_volume successfully ran.
[2025-01-09 23:41:35] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.576134095409512, BPM: 120', error=None, call_id='call_LJKjbpgM80Xtnjh32cNWJoUb', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='call_Yh0n9Jjalh9XIRFqNpt4YEPh', args='{"volume": 0.2}', name='change_music_volume')]
[2025-01-09 23:41:35] INFO | root: Generating text


The
The jazz
The jazz music
The jazz music has
The jazz music has started
The jazz music has started playing
The jazz music has started playing at
The jazz music has started playing at a
The jazz music has started playing at a low
The jazz music has started playing at a low volume
The jazz music has started playing at a low volume,
The jazz music has started playing at a low volume, and
The jazz music has started playing at a low volume, and the
The jazz music has started playing at a low volume, and the estimated
The jazz music has started playing at a low volume, and the estimated BPM
The jazz music has started playing at a low volume, and the estimated BPM for
The jazz music has started playing at a low volume, and the estimated BPM for the
The jazz music has started playing at a low volume, and the estimated BPM for the track
The jazz music has started playing at a low volume, and the estimated BPM for the track is
The jazz music has started playing at a low volume, and the estimat

[2025-01-09 23:41:37] DEBUG | Response: [TextResponse(text='The jazz music has started playing at a low volume, and the estimated BPM for the track is 120. Enjoy the music!')]


The jazz music has started playing at a low volume, and the estimated BPM for the track is 120. Enjoy the music!


In [16]:
streamed_response = openai_agent.stream_text_async(prompt)

async for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-09 23:41:37] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.\n\nCapabilities:\n- start_music: Start playing a random music with the given the genre.\n- change_music_volume: Change the music volume.\n- change_music: Change the music.\n- change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-09 23:41:37] INFO | root: Generating text
[2025-01-09 23:41:41] DEBUG | Response: [TextResponse(text='### Plan to Address the Requirement\n\n1. **Start Playing Jazz Music**: Use the `start_music` tool to begin playing jazz music.\n2. **Set Volume to Low**: Use the `change_music

{"ge
{"genre":
{"genre": "jazz
{"genre": "jazz"}
{"vo
{"volume"
{"volume": 0.2}


[2025-01-09 23:41:42] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='call_T6CrG8S9XDgJMdX1Iu9ygeP4'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='call_ffT70cX6wX7naOz9WhsnmUCj')]
[2025-01-09 23:41:42] INFO | Running tool: start_music
[2025-01-09 23:41:42] INFO | Tool: start_music successfully ran.
[2025-01-09 23:41:42] INFO | Running tool: change_music_volume
[2025-01-09 23:41:42] INFO | Tool: change_music_volume successfully ran.
[2025-01-09 23:41:42] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.4182263010115296, BPM: 120', error=None, call_id='call_T6CrG8S9XDgJMdX1Iu9ygeP4', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='call_ffT70cX6wX7naOz9WhsnmUCj', args='{"volume": 0.2}', name='change_music_volume')]
[2025-01-09 23:41:42] INFO | root: Generating text


The
The jazz
The jazz music
The jazz music has
The jazz music has started
The jazz music has started playing
The jazz music has started playing at
The jazz music has started playing at a
The jazz music has started playing at a low
The jazz music has started playing at a low volume
The jazz music has started playing at a low volume.
The jazz music has started playing at a low volume. The
The jazz music has started playing at a low volume. The current
The jazz music has started playing at a low volume. The current BPM
The jazz music has started playing at a low volume. The current BPM of
The jazz music has started playing at a low volume. The current BPM of the
The jazz music has started playing at a low volume. The current BPM of the music
The jazz music has started playing at a low volume. The current BPM of the music is
The jazz music has started playing at a low volume. The current BPM of the music is 
The jazz music has started playing at a low volume. The current BPM of the music i

[2025-01-09 23:41:43] DEBUG | Response: [TextResponse(text='The jazz music has started playing at a low volume. The current BPM of the music is 120. Enjoy!')]


The jazz music has started playing at a low volume. The current BPM of the music is 120. Enjoy!


## Other client

Other clients has the same interface.

In [17]:
# Azure OpenAI
azure_openai_agent = Agent(
    client=azure_openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=tools,
    # planning=True, # as needed
)

# Google AI Studio
google_agent = Agent(
    client=google_dev_client,
    model="gemini-2.0-flash-exp",
    temperature=0.0,
    tools=tools,
    # planning=True, # as needed
)

# Google Cloud VertexAI
vertexai_agent = Agent(
    client=vertexai_client,
    model="gemini-2.0-flash-exp",
    temperature=0.0,
    tools=tools,
    # planning=True, # as needed
)

# Anthropic Claude
claude_agent = Agent(
    client=anthropic_client,
    model="claude-3-5-sonnet-20240620",
    temperature=0.0,
    max_tokens=500,
    tools=tools,
    # planning=True, # as needed
)

# Claude on Amazon Bedrock
claude_bedrock_agent = Agent(
    client=claude_bedrock_client,
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0.0,
    max_tokens=500,
    tools=tools,
    # planning=True, # as needed
)

### Azure OpenAI

In [18]:
response = azure_openai_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-09 23:41:43] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:41:43] INFO | root: Generating text
[2025-01-09 23:41:44] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre":"jazz"}', call_id='call_PULYZxypbU5GB0vjkLCZdOlO')]
[2025-01-09 23:41:44] INFO | Running tool: start_music
[2025-01-09 23:41:44] INFO | Tool: start_music successfully ran.
[2025-01-09 23:41:44] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.6541270261036778, BPM: 120', error=None, call_id='call_PULYZxypbU5GB0vjkLCZdOlO', args='{"genre":"jazz"}', name='start_music')]
[2025-01-09 23:41:44] INFO | root: Generating text
[2025-01-09 23:41:45] DEBUG | Response: [ToolCallResponse(name='change_music_volume', args='{"volume":0.2}', call_id='call_oPeMfwWzWY3iX2Q1Y7tB8OFF')]
[2025-01-09 23:41:45] INFO | Running tool: change_music_volume
[2025-01-09 23:41:45] INFO | Tool: ch

I've started playing jazz music at a low volume. The current BPM (Beats Per Minute) is 120. Enjoy the music!


In [19]:
response = await azure_openai_agent.generate_text_async(prompt)

print(response.contents[0].text)

[2025-01-09 23:41:46] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:41:46] INFO | root: Generating text
[2025-01-09 23:41:47] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre":"jazz"}', call_id='call_jG5gJz0wuCXYW9i0SwkwZeys')]
[2025-01-09 23:41:47] INFO | Running tool: start_music
[2025-01-09 23:41:47] INFO | Tool: start_music successfully ran.
[2025-01-09 23:41:47] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.8506669430178333, BPM: 120', error=None, call_id='call_jG5gJz0wuCXYW9i0SwkwZeys', args='{"genre":"jazz"}', name='start_music')]
[2025-01-09 23:41:47] INFO | root: Generating text
[2025-01-09 23:41:47] DEBUG | Response: [ToolCallResponse(name='change_music_volume', args='{"volume":0.2}', call_id='call_lUzwGTlBYpApFjsedAEQnXck')]
[2025-01-09 23:41:47] INFO | Running tool: change_music_volume
[2025-01-09 23:41:47] INFO | Tool: ch

I've started playing jazz music at a low volume. The BPM of the music is 120. Enjoy!


In [20]:
streamed_response = azure_openai_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-09 23:41:48] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:41:48] INFO | root: Generating text


{"
{"genre
{"genre":"
{"genre":"j
{"genre":"jazz
{"genre":"jazz"}


[2025-01-09 23:41:49] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre":"jazz"}', call_id='call_YLosu260L7Xtk3InR0qbg9Le')]
[2025-01-09 23:41:49] INFO | Running tool: start_music
[2025-01-09 23:41:49] INFO | Tool: start_music successfully ran.
[2025-01-09 23:41:49] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.10836535452234974, BPM: 120', error=None, call_id='call_YLosu260L7Xtk3InR0qbg9Le', args='{"genre":"jazz"}', name='start_music')]
[2025-01-09 23:41:49] INFO | root: Generating text


{"
{"volume
{"volume":
{"volume":0
{"volume":0.
{"volume":0.2
{"volume":0.2}


[2025-01-09 23:41:50] DEBUG | Response: [ToolCallResponse(name='change_music_volume', args='{"volume":0.2}', call_id='call_Ff64AHSIfydqSHLpiJJG9fl4')]
[2025-01-09 23:41:50] INFO | Running tool: change_music_volume
[2025-01-09 23:41:50] INFO | Tool: change_music_volume successfully ran.
[2025-01-09 23:41:50] DEBUG | Prompt: [ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='call_Ff64AHSIfydqSHLpiJJG9fl4', args='{"volume":0.2}', name='change_music_volume')]
[2025-01-09 23:41:50] INFO | root: Generating text


I've
I've started
I've started playing
I've started playing jazz
I've started playing jazz music
I've started playing jazz music at
I've started playing jazz music at a
I've started playing jazz music at a low
I've started playing jazz music at a low volume
I've started playing jazz music at a low volume.
I've started playing jazz music at a low volume. The
I've started playing jazz music at a low volume. The current
I've started playing jazz music at a low volume. The current BPM
I've started playing jazz music at a low volume. The current BPM (
I've started playing jazz music at a low volume. The current BPM (Be
I've started playing jazz music at a low volume. The current BPM (Beats
I've started playing jazz music at a low volume. The current BPM (Beats Per
I've started playing jazz music at a low volume. The current BPM (Beats Per Minute
I've started playing jazz music at a low volume. The current BPM (Beats Per Minute)
I've started playing jazz music at a low volume. The current BP

[2025-01-09 23:41:50] DEBUG | Response: [TextResponse(text="I've started playing jazz music at a low volume. The current BPM (Beats Per Minute) is 120. Enjoy the music!")]


I've started playing jazz music at a low volume. The current BPM (Beats Per Minute) is 120. Enjoy the music!


In [21]:
streamed_response = azure_openai_agent.stream_text_async(prompt)

async for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-09 23:41:50] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:41:50] INFO | root: Generating text


{"
{"genre
{"genre":"
{"genre":"j
{"genre":"jazz
{"genre":"jazz"}


[2025-01-09 23:41:51] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre":"jazz"}', call_id='call_MXQGqJv0L0kMoNvim63NxlqP')]
[2025-01-09 23:41:51] INFO | Running tool: start_music
[2025-01-09 23:41:51] INFO | Tool: start_music successfully ran.
[2025-01-09 23:41:51] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.3140125409895157, BPM: 120', error=None, call_id='call_MXQGqJv0L0kMoNvim63NxlqP', args='{"genre":"jazz"}', name='start_music')]
[2025-01-09 23:41:51] INFO | root: Generating text


{"
{"volume
{"volume":
{"volume":0
{"volume":0.
{"volume":0.2
{"volume":0.2}


[2025-01-09 23:41:51] DEBUG | Response: [ToolCallResponse(name='change_music_volume', args='{"volume":0.2}', call_id='call_I5mRk788n7VurJJrZ1A1VJ4e')]
[2025-01-09 23:41:51] INFO | Running tool: change_music_volume
[2025-01-09 23:41:51] INFO | Tool: change_music_volume successfully ran.
[2025-01-09 23:41:51] DEBUG | Prompt: [ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='call_I5mRk788n7VurJJrZ1A1VJ4e', args='{"volume":0.2}', name='change_music_volume')]
[2025-01-09 23:41:51] INFO | root: Generating text


I've
I've started
I've started playing
I've started playing jazz
I've started playing jazz music
I've started playing jazz music at
I've started playing jazz music at a
I've started playing jazz music at a low
I've started playing jazz music at a low volume
I've started playing jazz music at a low volume.
I've started playing jazz music at a low volume. The
I've started playing jazz music at a low volume. The current
I've started playing jazz music at a low volume. The current BPM
I've started playing jazz music at a low volume. The current BPM (
I've started playing jazz music at a low volume. The current BPM (Be
I've started playing jazz music at a low volume. The current BPM (Beats
I've started playing jazz music at a low volume. The current BPM (Beats Per
I've started playing jazz music at a low volume. The current BPM (Beats Per Minute
I've started playing jazz music at a low volume. The current BPM (Beats Per Minute)
I've started playing jazz music at a low volume. The current BP

[2025-01-09 23:41:52] DEBUG | Response: [TextResponse(text="I've started playing jazz music at a low volume. The current BPM (Beats Per Minute) is 120. Enjoy the music!")]


I've started playing jazz music at a low volume. The current BPM (Beats Per Minute) is 120. Enjoy the music!


### Gemini on Google AI Studio

In [22]:
response = google_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-09 23:41:53] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:41:53] INFO | root: Generating text
[2025-01-09 23:41:53] DEBUG | Response: [TextResponse(text='I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?')]


I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?


In [23]:
response = await google_agent.generate_text_async(prompt)

print(response.contents[0].text)

[2025-01-09 23:41:53] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:41:53] INFO | root: Generating text
[2025-01-09 23:41:54] DEBUG | Response: [TextResponse(text='I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?')]


I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?


In [24]:
streamed_response = google_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-09 23:41:54] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:41:54] INFO | root: Generating text


I
I can play jazz music for you and set the volume to low. However, I
I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz
I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?

I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?



In [25]:
streamed_response = google_agent.stream_text_async(prompt)

async for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-09 23:41:55] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:41:55] INFO | root: Generating text


I
I can play jazz music for you and set the volume to low. However, I
I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz
I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?

I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?



### Gemini on VertexAI

In [26]:
response = vertexai_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-09 23:41:57] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:41:57] INFO | root: Generating text
[2025-01-09 23:41:59] DEBUG | Response: [TextResponse(text='I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?')]


I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?


In [27]:
response = await vertexai_agent.generate_text_async(prompt)

print(response.contents[0].text)

[2025-01-09 23:41:59] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:41:59] INFO | root: Generating text
[2025-01-09 23:42:00] DEBUG | Response: [TextResponse(text='I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?')]


I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?


In [28]:
streamed_response = vertexai_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-09 23:42:00] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:42:00] INFO | root: Generating text


I
I can play jazz music for you and set the volume to low. However, I
I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz
I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?

I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?



In [29]:
streamed_response = vertexai_agent.stream_text_async(prompt)

async for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-09 23:42:02] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:42:02] INFO | root: Generating text


I
I can play jazz music for you and set the volume to low. However, I
I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz
I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?

I can play jazz music for you and set the volume to low. However, I cannot detect the BPM of the music. Would you like to proceed with playing jazz music at a low volume?



### Claude on Anthropic

In [30]:
response = claude_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-09 23:42:03] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:42:03] INFO | root: Generating text
[2025-01-09 23:42:06] DEBUG | Response: [TextResponse(text="Certainly! I'd be happy to help you play some jazz music at a low volume. Let's break this down into steps:\n\n1. Start playing jazz music\n2. Lower the volume\n\nUnfortunately, I don't have a tool available to determine the BPM (Beats Per Minute) of the music. However, I can certainly help with playing the music and adjusting the volume.\n\nLet's start by playing jazz music and then adjusting the volume:"), ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='toolu_01XD9kXeMkoFMbZusVxXsh5A')]
[2025-01-09 23:42:06] INFO | Running tool: start_music
[2025-01-09 23:42:06] INFO | Tool: start_music successfully ran.
[2025-01-09 23:42:06] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume:

Excellent! The jazz music is now playing at a lower volume of 0.3.

Regarding your question about the BPM, I apologize that I don't have a tool to directly measure or report the BPM of the currently playing music. However, from the initial function result, we can see that the BPM of the jazz track that started playing is 120 BPM. Please note that this might change if the music changes to a different track.

Is there anything else you'd like me to adjust with the music, such as changing the equalizer settings or switching to a different genre?


In [31]:
response = await claude_agent.generate_text_async(prompt)

print(response.contents[0].text)

[2025-01-09 23:42:11] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:42:11] INFO | root: Generating text
[2025-01-09 23:42:14] DEBUG | Response: [TextResponse(text="Certainly! I'd be happy to help you play some jazz music at a low volume. Let's break this down into steps:\n\n1. Start playing jazz music\n2. Lower the volume\n\nUnfortunately, I don't have a tool available to determine the BPM (Beats Per Minute) of the music. However, I can certainly help with playing the music and adjusting the volume.\n\nLet's start by playing jazz music and then adjusting the volume:"), ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='toolu_014XsFDQqakvi3xesGcqnSwL')]
[2025-01-09 23:42:14] INFO | Running tool: start_music
[2025-01-09 23:42:14] INFO | Tool: start_music successfully ran.
[2025-01-09 23:42:14] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume:

Excellent! The jazz music is now playing at a lower volume of 0.3.

Regarding your question about the BPM, I apologize that I don't have a tool to determine that information. However, the system did provide some information when we started the music. According to the function results, the BPM of the current jazz track is 120.

Is there anything else you'd like me to adjust with the music, such as changing the equalizer settings or switching to a different genre?


In [32]:
streamed_response = claude_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-09 23:42:19] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:42:19] INFO | root: Generating text


Certainly! I'
Certainly! I'd be happy to help you play some
Certainly! I'd be happy to help you play some jazz music at a low volume.
Certainly! I'd be happy to help you play some jazz music at a low volume. Let's break this down into steps:

1. Start
Certainly! I'd be happy to help you play some jazz music at a low volume. Let's break this down into steps:

1. Start playing jazz music
2. Lower
Certainly! I'd be happy to help you play some jazz music at a low volume. Let's break this down into steps:

1. Start playing jazz music
2. Lower the volume

Unfortunately, I don't have a tool
Certainly! I'd be happy to help you play some jazz music at a low volume. Let's break this down into steps:

1. Start playing jazz music
2. Lower the volume

Unfortunately, I don't have a tool available to determine the BPM (
Certainly! I'd be happy to help you play some jazz music at a low volume. Let's break this down into steps:

1. Start playing jazz music
2. Lower the volume

Unfortunately, I don't ha

[2025-01-09 23:42:23] DEBUG | Response: [TextResponse(text="Certainly! I'd be happy to help you play some jazz music at a low volume. Let's break this down into steps:\n\n1. Start playing jazz music\n2. Lower the volume\n\nUnfortunately, I don't have a tool available to determine the BPM (Beats Per Minute) of the music. However, I can certainly help with playing the music and adjusting the volume.\n\nLet's start by playing jazz music and then adjusting the volume:"), ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='toolu_013mMXMxzJ6wgFNW2ebqDgx1')]
[2025-01-09 23:42:23] INFO | Running tool: start_music
[2025-01-09 23:42:23] INFO | Tool: start_music successfully ran.
[2025-01-09 23:42:23] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.1599102332895712, BPM: 120', error=None, call_id='toolu_013mMXMxzJ6wgFNW2ebqDgx1', args='{"genre": "jazz"}', name='start_music')]
[2025-01-09 23:42:23] INFO | root: Generating text


Great! The jazz
Great! The jazz music has started playing. Now
Great! The jazz music has started playing. Now, let's lower the volume.
Great! The jazz music has started playing. Now, let's lower the volume. Since you mentioned you prefer a low volume,
Great! The jazz music has started playing. Now, let's lower the volume. Since you mentioned you prefer a low volume, I'll set it to 0.2
Great! The jazz music has started playing. Now, let's lower the volume. Since you mentioned you prefer a low volume, I'll set it to 0.2 (on a scale of 0.
Great! The jazz music has started playing. Now, let's lower the volume. Since you mentioned you prefer a low volume, I'll set it to 0.2 (on a scale of 0.0 to 1.0,
Great! The jazz music has started playing. Now, let's lower the volume. Since you mentioned you prefer a low volume, I'll set it to 0.2 (on a scale of 0.0 to 1.0, where 0.0 is m
Great! The jazz music has started playing. Now, let's lower the volume. Since you mentioned you prefer a low volume, 

[2025-01-09 23:42:26] DEBUG | Response: [TextResponse(text="Great! The jazz music has started playing. Now, let's lower the volume. Since you mentioned you prefer a low volume, I'll set it to 0.2 (on a scale of 0.0 to 1.0, where 0.0 is muted and 1.0 is maximum volume):"), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='toolu_01JgJPF2dKbrMxdZ3p1CA43j')]
[2025-01-09 23:42:26] INFO | Running tool: change_music_volume
[2025-01-09 23:42:26] INFO | Tool: change_music_volume successfully ran.
[2025-01-09 23:42:26] DEBUG | Prompt: [ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='toolu_01JgJPF2dKbrMxdZ3p1CA43j', args='{"volume": 0.2}', name='change_music_volume')]
[2025-01-09 23:42:26] INFO | root: Generating text


Excellent! The jazz music is now playing at
Excellent! The jazz music is now playing at a lower volume of 0.2.
Excellent! The jazz music is now playing at a lower volume of 0.2.

Regarding your question about the BPM, I
Excellent! The jazz music is now playing at a lower volume of 0.2.

Regarding your question about the BPM, I apologize that I don't have a tool to
Excellent! The jazz music is now playing at a lower volume of 0.2.

Regarding your question about the BPM, I apologize that I don't have a tool to directly measure or report the BPM of the currently
Excellent! The jazz music is now playing at a lower volume of 0.2.

Regarding your question about the BPM, I apologize that I don't have a tool to directly measure or report the BPM of the currently playing music. However, from the initial
Excellent! The jazz music is now playing at a lower volume of 0.2.

Regarding your question about the BPM, I apologize that I don't have a tool to directly measure or report the BPM of the curre

[2025-01-09 23:42:28] DEBUG | Response: [TextResponse(text="Excellent! The jazz music is now playing at a lower volume of 0.2.\n\nRegarding your question about the BPM, I apologize that I don't have a tool to directly measure or report the BPM of the currently playing music. However, from the initial function result, we can see that the BPM of the jazz track that started playing is 120 BPM. Please note that this may vary if the music changes to a different track.\n\nIs there anything else you'd like me to adjust with the music, such as changing the equalizer settings or switching to a different genre?")]


Excellent! The jazz music is now playing at a lower volume of 0.2.

Regarding your question about the BPM, I apologize that I don't have a tool to directly measure or report the BPM of the currently playing music. However, from the initial function result, we can see that the BPM of the jazz track that started playing is 120 BPM. Please note that this may vary if the music changes to a different track.

Is there anything else you'd like me to adjust with the music, such as changing the equalizer settings or switching to a different genre?


In [33]:
streamed_response = claude_agent.stream_text_async(prompt)

async for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-09 23:42:28] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:42:28] INFO | root: Generating text


Certainly! I'
Certainly! I'd be happy to help you play some jazz music at a low
Certainly! I'd be happy to help you play some jazz music at a low volume. Let's break this down into steps
Certainly! I'd be happy to help you play some jazz music at a low volume. Let's break this down into steps:

1. Start playing jazz music
2
Certainly! I'd be happy to help you play some jazz music at a low volume. Let's break this down into steps:

1. Start playing jazz music
2. Lower the volume

Unfortunately, I don't have a tool
Certainly! I'd be happy to help you play some jazz music at a low volume. Let's break this down into steps:

1. Start playing jazz music
2. Lower the volume

Unfortunately, I don't have a tool available to determine the BPM (
Certainly! I'd be happy to help you play some jazz music at a low volume. Let's break this down into steps:

1. Start playing jazz music
2. Lower the volume

Unfortunately, I don't have a tool available to determine the BPM (Beats Per Minute) of the music

[2025-01-09 23:42:31] DEBUG | Response: [TextResponse(text="Certainly! I'd be happy to help you play some jazz music at a low volume. Let's break this down into steps:\n\n1. Start playing jazz music\n2. Lower the volume\n\nUnfortunately, I don't have a tool available to determine the BPM (Beats Per Minute) of the music. However, I can certainly help with playing the music and adjusting the volume.\n\nLet's start by playing jazz music and then adjusting the volume:"), ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='toolu_019HjTSPH8b8Av9zSWMvrn9K')]
[2025-01-09 23:42:31] INFO | Running tool: start_music
[2025-01-09 23:42:31] INFO | Tool: start_music successfully ran.
[2025-01-09 23:42:31] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.24923915231653593, BPM: 120', error=None, call_id='toolu_019HjTSPH8b8Av9zSWMvrn9K', args='{"genre": "jazz"}', name='start_music')]
[2025-01-09 23:42:31] INFO | root: Generating text


Great! The jazz
Great! The jazz music has started playing. Now, let's lower the
Great! The jazz music has started playing. Now, let's lower the volume. Since you mentioned you
Great! The jazz music has started playing. Now, let's lower the volume. Since you mentioned you prefer a low volume, I'll set it to
Great! The jazz music has started playing. Now, let's lower the volume. Since you mentioned you prefer a low volume, I'll set it to 0.2 (
Great! The jazz music has started playing. Now, let's lower the volume. Since you mentioned you prefer a low volume, I'll set it to 0.2 (on a scale of 0.
Great! The jazz music has started playing. Now, let's lower the volume. Since you mentioned you prefer a low volume, I'll set it to 0.2 (on a scale of 0.0 to 1.0,
Great! The jazz music has started playing. Now, let's lower the volume. Since you mentioned you prefer a low volume, I'll set it to 0.2 (on a scale of 0.0 to 1.0, where 0.0 is m
Great! The jazz music has started playing. Now, let's lower

[2025-01-09 23:42:34] DEBUG | Response: [TextResponse(text="Great! The jazz music has started playing. Now, let's lower the volume. Since you mentioned you prefer a low volume, I'll set it to 0.2 (on a scale of 0.0 to 1.0, where 0.0 is muted and 1.0 is maximum volume):"), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='toolu_014doCQwofvx5rnbNoVSLxTp')]
[2025-01-09 23:42:34] INFO | Running tool: change_music_volume
[2025-01-09 23:42:34] INFO | Tool: change_music_volume successfully ran.
[2025-01-09 23:42:34] DEBUG | Prompt: [ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='toolu_014doCQwofvx5rnbNoVSLxTp', args='{"volume": 0.2}', name='change_music_volume')]
[2025-01-09 23:42:34] INFO | root: Generating text


Excellent! The jazz music is now playing at
Excellent! The jazz music is now playing at a lower volume of 0.2.
Excellent! The jazz music is now playing at a lower volume of 0.2.

Regarding the BPM, while
Excellent! The jazz music is now playing at a lower volume of 0.2.

Regarding the BPM, while I don't have a tool to measure
Excellent! The jazz music is now playing at a lower volume of 0.2.

Regarding the BPM, while I don't have a tool to measure it directly, the system provide
Excellent! The jazz music is now playing at a lower volume of 0.2.

Regarding the BPM, while I don't have a tool to measure it directly, the system provided some information when we started the music. According
Excellent! The jazz music is now playing at a lower volume of 0.2.

Regarding the BPM, while I don't have a tool to measure it directly, the system provided some information when we started the music. According to that information, the current
Excellent! The jazz music is now playing at a lower volume of

[2025-01-09 23:42:37] DEBUG | Response: [TextResponse(text="Excellent! The jazz music is now playing at a lower volume of 0.2.\n\nRegarding the BPM, while I don't have a tool to measure it directly, the system provided some information when we started the music. According to that information, the current jazz track has a BPM of 120.\n\nTo summarize:\n1. Jazz music is now playing\n2. The volume has been set to a low level (0.2 out of 1.0)\n3. The current track has a BPM of 120\n\nIs there anything else you'd like me to adjust with the music, or do you have any other questions?")]


Excellent! The jazz music is now playing at a lower volume of 0.2.

Regarding the BPM, while I don't have a tool to measure it directly, the system provided some information when we started the music. According to that information, the current jazz track has a BPM of 120.

To summarize:
1. Jazz music is now playing
2. The volume has been set to a low level (0.2 out of 1.0)
3. The current track has a BPM of 120

Is there anything else you'd like me to adjust with the music, or do you have any other questions?


# Structured output using tool calling

## Response schema as a tool

Structured output in langrila is just tool calling (please refer to [01.introduction.ipynb](./01.introduction.ipynb)), so you can get structured response even if you are using tool calling.

In [34]:
from pydantic import BaseModel, Field


class RessponseSchema(BaseModel):
    genre: MusicGenre = Field(
        ...,
        description="The genre of music.",
    )
    bpm: int = Field(
        ...,
        description="The beats per minute of the music.",
        ge=50,
        le=200,
    )
    volume: float = Field(
        ...,
        description="The volume level of the music.",
        ge=0,
        le=1,
    )
    equalizer_settings: EqualizerSettings = Field(
        ...,
        description="The equalizer settings.",
    )

In [35]:
openai_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=tools,
    # planning=True,
    response_schema_as_tool=RessponseSchema,
)

In [36]:
response = openai_agent.generate_text(prompt)

RessponseSchema.model_validate_json(response.contents[0].text)

[2025-01-09 23:42:37] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:42:37] INFO | root: Generating text
[2025-01-09 23:42:38] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre":"jazz"}', call_id='call_QgLHEKuNhBOt4ZO9KV4Tdv9b')]
[2025-01-09 23:42:38] INFO | Running tool: start_music
[2025-01-09 23:42:38] INFO | Tool: start_music successfully ran.
[2025-01-09 23:42:38] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.6994507065665946, BPM: 120', error=None, call_id='call_QgLHEKuNhBOt4ZO9KV4Tdv9b', args='{"genre":"jazz"}', name='start_music')]
[2025-01-09 23:42:38] INFO | root: Generating text
[2025-01-09 23:42:39] DEBUG | Response: [ToolCallResponse(name='change_music_volume', args='{"volume":0.2}', call_id='call_tp81IJcd4eeR9ZWkqmWiRvB9')]
[2025-01-09 23:42:39] INFO | Running tool: change_music_volume
[2025-01-09 23:42:39] INFO | Tool: ch

RessponseSchema(genre=<MusicGenre.jazz: 'jazz'>, bpm=120, volume=0.2, equalizer_settings=EqualizerSettings(hz_100=0, hz_200=0, hz_400=0, hz_800=0, hz_1600=0, hz_3200=0, hz_6400=0))

Same for other clients, so examples will be omitted.

## Native response schema

If you want to use native structured output, you can specify the original parameters of LLM provider API. For example, for OpenAI API, you can use `response_format` parameter supported by OpenAI API natively. Please be aware of limitations of the API in this case.

In [37]:
from pydantic import BaseModel, Field


# default value and range limitation are not supported by OpenAI API's response format
class NativeEqualizerSettings(BaseModel):
    hz_100: int = Field(
        ...,
        description="The 100 Hz band in dB. It should be between -12 and 12. Default is 0.",
    )
    hz_200: int = Field(
        ...,
        description="The 200 Hz band in dB. It should be between -12 and 12. Default is 0.",
    )
    hz_400: int = Field(
        ...,
        description="The 400 Hz band in dB. It should be between -12 and 12. Default is 0.",
    )
    hz_800: int = Field(
        ...,
        description="The 800 Hz band in dB. It should be between -12 and 12. Default is 0.",
    )
    hz_1600: int = Field(
        ...,
        description="The 1600 Hz band in dB. It should be between -12 and 12. Default is 0.",
    )
    hz_3200: int = Field(
        ...,
        description="The 3200 Hz band in dB. It should be between -12 and 12. Default is 0.",
    )
    hz_6400: int = Field(
        ...,
        description="The 4000 Hz band in dB. It should be between -12 and 12. Default is 0.",
    )


class NativeResponseSchema(BaseModel):
    genre: MusicGenre = Field(..., description="The genre of music to play.")
    bpm: int = Field(
        ...,
        description="The BPM of the music.",
    )
    volume: float = Field(
        ...,
        description="The volume level to set the music to. It should be between 0.0 and 1.0.",
    )
    equalizer_settings: NativeEqualizerSettings = Field(
        ..., description="The equalizer settings to set."
    )

Re-definition to use native response format.

In [44]:
def change_equalizer_settings_native(settings: NativeEqualizerSettings) -> str:
    """
    Change the equalizer settings of the music.

    Parameters
    ----------
    settings : NativeEqualizerSettings
        The new equalizer settings to set.

    Returns
    ----------
    str
        A message indicating that the equalizer settings have been changed.
    """
    return f"Changing equalizer settings to {settings.model_dump()}"

In [45]:
tools_native = [start_music, change_music_volume, change_music, change_equalizer_settings_native]

In [46]:
openai_agent_native = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    tools=tools_native,
    response_format=NativeResponseSchema,
)

In [47]:
response = openai_agent_native.generate_text(prompt)

NativeResponseSchema.model_validate_json(response.contents[0].text)

[2025-01-09 23:43:24] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:43:24] INFO | root: Generating text
[2025-01-09 23:43:24] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre":"jazz"}', call_id='call_N4xJHbwJaT7702QWqHsbmP2e')]
[2025-01-09 23:43:24] INFO | Running tool: start_music
[2025-01-09 23:43:24] INFO | Tool: start_music successfully ran.
[2025-01-09 23:43:24] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.3360032960245172, BPM: 120', error=None, call_id='call_N4xJHbwJaT7702QWqHsbmP2e', args='{"genre":"jazz"}', name='start_music')]
[2025-01-09 23:43:25] INFO | root: Generating text
[2025-01-09 23:43:25] DEBUG | Response: [ToolCallResponse(name='change_music_volume', args='{"volume":0.1}', call_id='call_UxUbY7H4DASLShw3NSyJRz1T')]
[2025-01-09 23:43:25] INFO | Running tool: change_music_volume
[2025-01-09 23:43:25] INFO | Tool: ch

NativeResponseSchema(genre=<MusicGenre.jazz: 'jazz'>, bpm=79, volume=0.1, equalizer_settings=NativeEqualizerSettings(hz_100=0, hz_200=0, hz_400=0, hz_800=0, hz_1600=0, hz_3200=0, hz_6400=0))

In [48]:
response

Response(type='Response', role='assistant', contents=[TextResponse(text='{"genre":"jazz","bpm":79,"volume":0.1,"equalizer_settings":{"hz_100":0,"hz_200":0,"hz_400":0,"hz_800":0,"hz_1600":0,"hz_3200":0,"hz_6400":0}}')], usage=<langrila.core.usage.NamedUsage object at 0x7f0ca8606890>, raw=ParsedChatCompletion[NativeResponseSchema](id='chatcmpl-Ano2p7Vw9flxgB2deUkeXQhrucIIj', choices=[ParsedChoice[NativeResponseSchema](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[NativeResponseSchema](content='{"genre":"jazz","bpm":79,"volume":0.1,"equalizer_settings":{"hz_100":0,"hz_200":0,"hz_400":0,"hz_800":0,"hz_1600":0,"hz_3200":0,"hz_6400":0}}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], parsed=NativeResponseSchema(genre=<MusicGenre.jazz: 'jazz'>, bpm=79, volume=0.1, equalizer_settings=NativeEqualizerSettings(hz_100=0, hz_200=0, hz_400=0, hz_800=0, hz_1600=0, hz_3200=0, hz_6400=0))))], created=1736433807, model='gpt-4o-mini-202

# Dependency injection

While tools can be passed to an Agent directly as functions, using the Tool class allows for more flexible control. One example of this flexibility is dependency injection. 

Standard tool calling requires the LLM to generate all arguments specified as 'required'. For arguments not specified as 'required', either a default value or a value generated by the LLM is used.

With the Tool class, you can use a 'context' argument to separate parameters into those generated by the LLM and those injected from an external source. This notebook demonstrates an example of this approach.

Here is an example of tool invokes an other agent. 

In [49]:
def run_music_agent(
    agent: Agent,  # type: ignore
    agent_name: str,
    instruction: str,
) -> str:
    """
    This function is used to run the agent which has the capabilities to manage the music settings.
    Agent can use the following tools:

    Tools:
    start_music: Turn on the music. The genre, BPM, and volume are randomly selected.
    change_bpm: Change the BPM of the music.
    change_music: Change the music genre and BPM.

    Parameters
    ----------
    agent : Agent
        The agent instance.
    agent_name : str
        The name of the agent.
    instruction : str
        The detail and specific instruction to the agent, including the plan to get answer.

    Returns
    ----------
    str
        The response from the agent.
    """
    if not isinstance(agent, Agent):
        raise ValueError(
            "Subagent must be an instance of Agent class. "
            "Please provide the correct agent instance."
        )

    return agent.generate_text(instruction, name=agent_name).contents[0].text  # type: ignore

To show tha example of dependency injection, two agents are defined; `music_agent` for managing music setting, and another agent calls `music_agent`.

In [50]:
# The agent for managing music settings
music_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=[start_music, change_music_volume, change_music, change_equalizer_settings],
)

# The main agent to call the music agent
agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    tools=[
        Tool(tool=run_music_agent, context={"agent": music_agent, "agent_name": "music_agent"}),
    ],
)

The tool's parameters specified in the context arguments of Tool instance are injected externally. In this case, `agent` argument and `agent_name` argument of the run_music_agent tool are injected.

In [51]:
agent.tools[0].context

{'agent': Agent(name=root), 'agent_name': 'music_agent'}

In [52]:
response = agent.generate_text(prompt)

[2025-01-09 23:43:35] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:43:35] INFO | root: Generating text
[2025-01-09 23:43:36] DEBUG | Response: [ToolCallResponse(name='run_music_agent', args='{"instruction":"Start playing jazz music at low volume and provide the BPM."}', call_id='call_r8ZN1kh2VYamFBbu0HqzLGJi')]
[2025-01-09 23:43:36] INFO | Running tool: run_music_agent
[2025-01-09 23:43:36] DEBUG | Prompt: [TextPrompt(text='Start playing jazz music at low volume and provide the BPM.')]
[2025-01-09 23:43:36] INFO | music_agent: Generating text
[2025-01-09 23:43:37] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='call_I63XQZ857VmwZ7gW3sHKXzJM'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='call_EI8al0eehamvFE9oUetTLKk4')]
[2025-01-09 23:43:37] INFO | Running tool: start_music
[2025-01-09 23:43:37] INFO | Tool: st

As you can see in the log messages above, the agent instance invoked music_agent, but the parameters created by the agent to call `music_agent` is only `instruction` argument while `agent` argument and `agent_name` are injected. The arguments are also validated when the tool is called. If you passed wrong value for a context parameter, the error will occur when run tool.

In [53]:
from langrila import AgentConfig

# Define not to retry the request even if an error occurs
agent_config = AgentConfig(
    max_error_retries=1,
)

# The main agent to call the music agent
agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    tools=[
        Tool(
            tool=run_music_agent,
            # wrong for agent argument below
            context={"agent": "wrong_value", "agent_name": "music_agent"},
        ),
    ],
    agent_config=agent_config,
)

In [54]:
response = agent.generate_text(prompt)

[2025-01-09 23:43:39] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:43:39] INFO | root: Generating text
[2025-01-09 23:43:40] DEBUG | Response: [ToolCallResponse(name='run_music_agent', args='{"instruction":"Start playing jazz music at a low volume and provide the BPM."}', call_id='call_VydLOjWCC8FDIym2Gd4XT1H3')]
[2025-01-09 23:43:40] INFO | Running tool: run_music_agent
[2025-01-09 23:43:40] ERROR | Validation error occurred while running tool. Retrying...
Tool: run_music_agent
Attribute: agent
Input value: wrong_value
ValidationError: Input should be an instance of Agent


RetryLimitExceededError: Retry limit exceeded. Please try again or change the request parameters.

Actually the tools calling other agent such as this are dynamically created in the Agent class if you pass the subagents parameter of the agent, but in this notebook, dared to define a tool as an example and passed it to the Agent class as a tool. For multi-agent example, Please refer to [03.multi_agent.ipynb](./03.multi_agent.ipynb)

# Tool result serializer

Tool class accepts `serializer` parameter that is callable object to convert any value to string. `str` is used in default, but you can also use custom serializer. This capability allows us to reuse the tools implemented in daily development by only implementing serializer. Let's change the example above using custom serializer.

In [55]:
import json

from langrila import Response


def run_music_agent(
    agent: Agent,  # type: ignore
    agent_name: str,
    instruction: str,
) -> Response:
    """
    This function is used to run the agent which has the capabilities to manage the music settings.
    Agent can use the following tools:

    Tools:
    start_music: Turn on the music. The genre, BPM, and volume are randomly selected.
    change_bpm: Change the BPM of the music.
    change_music: Change the music genre and BPM.

    Parameters
    ----------
    agent : Agent
        The agent instance.
    agent_name : str
        The name of the agent.
    instruction : str
        The detail and specific instruction to the agent, including the plan to get answer.

    Returns
    ----------
    Response
        The response from the agent.
    """
    if not isinstance(agent, Agent):
        raise ValueError(
            "Subagent must be an instance of Agent class. "
            "Please provide the correct agent instance."
        )

    return agent.generate_text(instruction, name=agent_name)


def response_serializer(response: Response) -> str:
    """
    Serialize the response to a string.

    Parameters
    ----------
    response : Response
        The response from the agent.

    Returns
    ----------
    str
        The serialized response.
    """
    return json.dumps(response.model_dump(include={"role", "contents", "name"}), ensure_ascii=False)

In [56]:
from langrila import AgentConfig

# Define not to retry the request even if an error occurs
agent_config = AgentConfig(
    max_error_retries=1,
)

# The main agent to call the music agent
agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    tools=[
        Tool(
            tool=run_music_agent,
            context={"agent": music_agent, "agent_name": "music_agent"},
            serializer=response_serializer,  # serializer
        ),
    ],
)

In [57]:
response = agent.generate_text(prompt)

[2025-01-09 23:43:54] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume. Also, I want to know the BPM.')]
[2025-01-09 23:43:54] INFO | root: Generating text
[2025-01-09 23:43:55] DEBUG | Response: [ToolCallResponse(name='run_music_agent', args='{"instruction":"Start playing jazz music at a low volume and provide the BPM."}', call_id='call_YDXxR8AgF5E2g93AWbgbn7v4')]
[2025-01-09 23:43:55] INFO | Running tool: run_music_agent
[2025-01-09 23:43:55] DEBUG | Prompt: [TextPrompt(text='Start playing jazz music at a low volume and provide the BPM.')]
[2025-01-09 23:43:55] INFO | music_agent: Generating text
[2025-01-09 23:43:56] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='call_ABAomrBBFaNSEqyTHyR61vgE'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.3}', call_id='call_oF5g0FM4KP0B6IfhpOyyVcyv')]
[2025-01-09 23:43:56] INFO | Running tool: start_music
[2025-01-09 23:43:56] INFO | Tool

ToolUsePrompt of the `run_music_agent` in the log message is changed to json string as expected.